In [0]:
use catalog IDENTIFIER(:catalog);
use schema IDENTIFIER(:schema);

In [0]:
CREATE OR REPLACE FUNCTION generate_viewer_url(modality STRING, viewer_host STRING, StudyInstanceUID STRING) RETURNS STRING
RETURN
CASE 
    WHEN modality = 'CT' THEN concat('<a target="_blank" rel="noopener noreferrer" href="',viewer_host,'/ohif/monai-label?StudyInstanceUIDs=',StudyInstanceUID,'">view</a>')
    ELSE concat('<a target="_blank" rel="noopener noreferrer" href="',viewer_host,'/ohif/viewer?StudyInstanceUIDs=',StudyInstanceUID,'">view</a>') 
END;

In [0]:
select 
  generate_viewer_url(
    meta:['00080060'].Value[0],
    'https://e2-demo-field-eng.cloud.databricks.com',
    meta:['0020000D'].Value[0]
  ) as viewer_url
from hls_radiology.tcia.object_catalog;

In [0]:
%python
-- cohort builder for Pixels Dashboard
view_sql = """
CREATE OR REPLACE VIEW cohort_builder_pixels AS
  with base (

  select
    meta:['00100010'].Value[0].Alphabetic as PatientName,
    meta:['00100020'].Value[0] PatientID,
    to_date(meta:['00100030'].Value[0],"yyyyMMdd") as PatientBirthDate,
    cast(left(meta:['00101010'].Value[0],3) as int) as PatientAge,
    meta:['00100040'].Value[0] PatientSex,
    to_date(meta:['00080020'].Value[0],"yyyyMMdd") as StudyDate,
    meta:['00080030'].Value[0] StudyTime,
    meta:['00080050'].Value[0] AccessionNumber,
    meta:['00080060'].Value[0] Modality,
    meta:['00080008'].Value[0] ImageType,
    meta:['00081030'].Value[0] StudyDescription,
    meta:['0008103E'].Value[0] SeriesDescription,
    meta:['0020000D'].Value[0] StudyInstanceUID,
    meta:['0020000E'].Value[0] SeriesInstanceUID,
    meta:['00200010'].Value[0] StudyID,
    meta:['00200011'].Value[0] SeriesNumber,
    meta:['00200013'].Value[0] InstanceNumber,
    meta:['00180015'].Value[0] BodyPartExamined,
    meta:['00181030'].Value[0] ProtocolName,
    meta:['00080070'].Value[0] Manufacturer,
    meta:['00080080'].Value[0] InstitutionName,
    meta:['00080081'].Value[0] InstitutionAddress,
    meta:['0018700A'].Value[0] DetectorManufacturerName,
    meta:['0018700B'].Value[0] DetectorManufacturersModelName,
    meta:['00189024'].Value[0] MRTransmitCoilManufacturerName,
    meta:['00189025'].Value[0] ReceiveCoilManufacturerName,
    meta:['00120063'].Value[0] DeidentificationMethod,
    meta:['0008001C'].Value[0] SyntheticData,
    meta:['00120064'].Value[0] CodingSchemeVersion,
    meta:['00282110'].Value[0] LossyImageCompression,
    meta:['00131010'].Value[0] Creator,
    meta:['pixel_hash'] pixel_hash,
    meta:['hash'] hash,
    path,
    length,
    modificationTime,
    meta
  from identifier(:table)
  )
  -- CASE expr {WHEN opt1 THEN res1} [...] [ELSE def] END
  -- CASE {WHEN cond1 THEN res1} [...] [ELSE def] END
  SELECT 
      hls_radiology.common.generate_viewer_url(
        modality,
        :viewer_host,
        SeriesInstanceUID
      ) as url
      ,*
  FROM base
"""
spark.sql(view_sql, {"table": f"{table}")